In [1]:
import csv
import json
import pandas as pd
from datetime import datetime
import numpy as np

from collections import Counter

outputFolders = "../data_2019_processed/"

fileRaw = "allData_filtred.csv"
fileNumeric = "-numeric.csv"
fileNumericNormalized = "-numeric-normalized.csv"
fileNumericClassification = "-numeric-classification.csv"
fileNumericClassificationNormalized = "-classification-numeric-normalized.csv"

In [2]:
def convert_to_decibels(value):
    if(value > 0.0):
        return 20 * (np.log(32768) - np.log(value/32768) )
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0

def treat_data_to_0_1(value):
    if(value > 0.0):
        return 1.0
        #return 20 * ( math.log(32768/value) )
        #return 20 * (np.log(value)) 
    else:
        return 0.0
    
def transform_data(dataframe):
    # convert data and show the new info
    # activity
    dataframe['activity'] = dataframe['activity'].replace(['Unknown'], '1')
    dataframe['activity'] = dataframe['activity'].replace(['Still'], '2')
    dataframe['activity'] = dataframe['activity'].replace(['Tilting'], '3')
    dataframe['activity'] = dataframe['activity'].replace(['Exercise'], '4')
    dataframe['activity'] = dataframe['activity'].replace(['In vehicle'], '5')
    # class
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    # day_of_week
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Tuesday'], '1')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Wednesday'], '2')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Thursday'], '3')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Friday'], '4')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Saturday'], '5')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Sunday'], '6')
    dataframe['day_of_week'] = dataframe['day_of_week'].replace(['Monday'], '7')
    # phone_lock
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['not locked'], '0')
    dataframe['phone_lock'] = dataframe['phone_lock'].replace(['locked'], '1')
    # location
    dataframe['location'] = dataframe['location'].replace(['Other'], '1')
    dataframe['location'] = dataframe['location'].replace(['University'], '2')
    dataframe['location'] = dataframe['location'].replace(['House'], '3')
    
    # timestamp
    if 'timestamp2' in dataframe.columns:
        dataframe['timestamp_text'] = dataframe['timestamp']
        dataframe['timestamp'] = dataframe['timestamp2']
        del dataframe['timestamp2']
    if 'outside_range' in dataframe.columns:
        del dataframe['outside_range']
    if 'time_sleeping' in dataframe.columns:
        del dataframe['time_sleeping']
    if 'wakeup_time' in dataframe.columns:
        del dataframe['wakeup_time']
    if 'sleep_time' in dataframe.columns:
        del dataframe['sleep_time']
    if 'distanceHome' in dataframe.columns:
        del dataframe['distanceHome']    
    if 'showActuation' in dataframe.columns:
        del dataframe['showActuation'] 
    #dataframe['timestamp'] = dataframe['timestamp'].astype('datetime64[ns]')
    #dataframe['timestamp'] = dataframe['timestamp'].astype('int64')
    #dataframe['timestamp'] = datetime.strptime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S').timestamp() * 1000
    #dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'],'%Y-%m-%d %H:%M:%S') * 1000
    #dataframe['timestamp'] = dataframe['timestamp'].dt.second()
    # data_type
    #dataframe['activity'] = dataframe['activity'].astype(np.integer) 
    #dataframe['class'] = dataframe['class'].astype(np.integer) 
    #dataframe['day_of_week'] = dataframe['day_of_week'].astype(np.integer) 
    #dataframe['phone_lock'] = dataframe['phone_lock'].astype(np.integer) 
    #dataframe['location'] = dataframe['location'].astype(np.integer) 
    dataframe['sound'] = dataframe['sound'].apply(convert_to_decibels)
    dataframe['proximity'] = dataframe['proximity'].apply(treat_data_to_0_1)
    dataframe['activity'] = dataframe['activity'].astype('int32')
    dataframe['class'] = dataframe['class'].astype('int32')
    dataframe['day_of_week'] = dataframe['day_of_week'].astype('int32')
    dataframe['phone_lock'] = dataframe['phone_lock'].astype('int32') 
    dataframe['location'] = dataframe['location'].astype('int32')
    dataframe['minutes_day'] = dataframe['minutes_day'].astype('float32')
    dataframe['time_to_next_alarm'] = dataframe['time_to_next_alarm'].astype('float32')
    dataframe['sound'] = dataframe['sound'].astype('float32')
    dataframe['light'] = dataframe['light'].astype('float32')
    dataframe['proximity'] = dataframe['proximity'].astype('float32')
    dataframe['timestamp'] = dataframe['timestamp'].astype('float32')
    
    temp = dataframe['class'].copy()
    del dataframe['class']
    dataframe['class'] = temp
    #return dataframe
    return dataframe

In [3]:
def transform_normalize_data(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name != "class" and feature_name != "timestamp_text":
            max_value = df[feature_name].max()
            if(feature_name == "timestamp"):
                max_value = 1924905600.0 #2030-12-31 00:00:00  
            if(feature_name == "day_of_week"):
                max_value = 7            
            if(feature_name == "sound"):
                max_value = 403 # overhal higher value            
            if(feature_name == "location"):
                max_value = 3
            if(feature_name == "activity"):
                max_value = 5
            if(feature_name == "minutes_day"):
                max_value = 1439 # 24 horas
            if(feature_name == "time_to_next_alarm"):
                max_value = 604800000.0 # 2.8 years in ms
            min_value = df[feature_name].min()
            print(feature_name+" "+str(min_value)+" "+str(max_value))
            if feature_name != "showActuation":
                result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
            if(max_value == 0 and min_value == 0):
                result[feature_name] = 0.0
    return result

In [4]:
def transform_data_type(dataset):
    # diminui o tamanho do tipo de dado (float32 é mais que o suficiente)
    dataset['activity'] = dataset['activity'].astype('float32')
    dataset['location'] = dataset['location'].astype('float32')
    dataset['timestamp'] = dataset['timestamp'].astype('float32')
    dataset['day_of_week'] = dataset['day_of_week'].astype('float32')
    dataset['light'] = dataset['light'].astype('float32')
    dataset['phone_lock'] = dataset['phone_lock'].astype('float32')
    dataset['proximity'] = dataset['proximity'].astype('float32')
    dataset['sound'] = dataset['sound'].astype('float32')
    dataset['time_to_next_alarm'] = dataset['time_to_next_alarm'].astype('float32')
    
    return dataset

In [5]:
def transform_output_to_numeric(dataframe):
    dataframe['class'] = dataframe['class'].replace(['awake'], '0')
    dataframe['class'] = dataframe['class'].replace(['asleep'], '1')
    
    dataframe['class'] = dataframe['class'].astype('int32')
    
    return dataframe

In [6]:
def transform_output_to_str(dataframe):
    dataframe['class'] = dataframe['class'].apply(str)
    #print(dataframe['class'])
    dataframe['class'] = dataframe['class'].replace(to_replace='0',value='awake')
    dataframe['class'] = dataframe['class'].replace(to_replace='1',value='asleep')
    
    return dataframe

# Load all data file

In [7]:
dataset = pd.read_csv(outputFolders+fileRaw)

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  object 
 1   location            542339 non-null  object 
 2   distanceHome        542339 non-null  int64  
 3   showActuation       542339 non-null  bool   
 4   timestamp           542339 non-null  object 
 5   time_to_next_alarm  542339 non-null  float64
 6   sound               542339 non-null  int64  
 7   proximity           542339 non-null  int64  
 8   phone_lock          542339 non-null  object 
 9   light               542339 non-null  int64  
 10  day_of_week         542339 non-null  object 
 11  class               542339 non-null  object 
 12  timestamp2          542339 non-null  int64  
 13  minutes_day         542339 non-null  int64  
 14  sleep_time          542339 non-null  int64  
 15  wakeup_time         542339 non-nul

In [8]:
dataset

,activity,location,distanceHome,showActuation,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,class,timestamp2,minutes_day,sleep_time,wakeup_time,time_sleeping
0,Unknown,Other,0,False,2018-05-15 14:20:45,594000000.0,0,5,not locked,37,Tuesday,awake,1526390445,860,1020,660,64808
1,Unknown,Other,0,False,2018-05-15 14:20:45,594000000.0,0,5,not locked,37,Tuesday,awake,1526390445,860,1020,660,64808
2,Still,University,0,False,2018-05-15 14:21:15,593940000.0,5517,5,not locked,35,Tuesday,awake,1526390475,861,1020,660,64808
3,Still,University,0,False,2018-05-15 14:21:45,593940000.0,5517,5,not locked,307,Tuesday,awake,1526390505,861,1020,660,64808
4,Still,University,0,False,2018-05-15 14:22:15,593880000.0,5809,5,not locked,42,Tuesday,awake,1526390535,862,1020,660,64808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542334,Still,House,0,False,2018-06-13 12:14:37,59999.0,0,5,locked,8,Wednesday,awake,1528888477,734,0,0,0
542335,Still,House,0,False,2018-06-13 12:18:08,420000.0,0,5,locked,11,Wednesday,awake,1528888688,738,0,0,0
542336,Still,House,0,False,2018-06-13 12:19:08,359999.0,0,5,locked,11,Wednesday,awake,1528888748,739,0,0,0
542337,Still,House,0,False,2018-06-13 12:19:38,359999.0,0,5,locked,12,Wednesday,awake,1528888778,739,0,0,0


In [9]:
# AllData Numeric to classification
datasetNumericClassification = transform_data(dataset)

datasetNumericClassification.info()
maxT = datasetNumericClassification['timestamp'].max()
minT = datasetNumericClassification['timestamp'].min()
print(maxT,minT)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  int32  
 1   location            542339 non-null  int32  
 2   timestamp           542339 non-null  float32
 3   time_to_next_alarm  542339 non-null  float32
 4   sound               542339 non-null  float32
 5   proximity           542339 non-null  float32
 6   phone_lock          542339 non-null  int32  
 7   light               542339 non-null  float32
 8   day_of_week         542339 non-null  int32  
 9   minutes_day         542339 non-null  float32
 10  timestamp_text      542339 non-null  object 
 11  class               542339 non-null  int32  
dtypes: float32(6), int32(5), object(1)
memory usage: 26.9+ MB
1528892300.0 1526043900.0


In [10]:
datasetNumericClassification = transform_data_type(datasetNumericClassification)

print(datasetNumericClassification.info())

datasetNumericClassification

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  float32
 1   location            542339 non-null  float32
 2   timestamp           542339 non-null  float32
 3   time_to_next_alarm  542339 non-null  float32
 4   sound               542339 non-null  float32
 5   proximity           542339 non-null  float32
 6   phone_lock          542339 non-null  float32
 7   light               542339 non-null  float32
 8   day_of_week         542339 non-null  float32
 9   minutes_day         542339 non-null  float32
 10  timestamp_text      542339 non-null  object 
 11  class               542339 non-null  int32  
dtypes: float32(10), int32(1), object(1)
memory usage: 26.9+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
1,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
2,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,35.0,1.0,861.0,2018-05-15 14:21:15,0
3,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,307.0,1.0,861.0,2018-05-15 14:21:45,0
4,2.0,2.0,1.526391e+09,593880000.0,242.545029,1.0,0.0,42.0,1.0,862.0,2018-05-15 14:22:15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,2.0,3.0,1.528888e+09,59999.0,0.000000,1.0,1.0,8.0,2.0,734.0,2018-06-13 12:14:37,0
542335,2.0,3.0,1.528889e+09,420000.0,0.000000,1.0,1.0,11.0,2.0,738.0,2018-06-13 12:18:08,0
542336,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,11.0,2.0,739.0,2018-06-13 12:19:08,0
542337,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,12.0,2.0,739.0,2018-06-13 12:19:38,0


In [11]:
# save files
datasetNumericClassification.to_csv(outputFolders+"allData"+fileNumericClassification, sep=',', encoding='utf-8', index=False)

datasetNumericClassification

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
1,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
2,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,35.0,1.0,861.0,2018-05-15 14:21:15,0
3,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,307.0,1.0,861.0,2018-05-15 14:21:45,0
4,2.0,2.0,1.526391e+09,593880000.0,242.545029,1.0,0.0,42.0,1.0,862.0,2018-05-15 14:22:15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,2.0,3.0,1.528888e+09,59999.0,0.000000,1.0,1.0,8.0,2.0,734.0,2018-06-13 12:14:37,0
542335,2.0,3.0,1.528889e+09,420000.0,0.000000,1.0,1.0,11.0,2.0,738.0,2018-06-13 12:18:08,0
542336,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,11.0,2.0,739.0,2018-06-13 12:19:08,0
542337,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,12.0,2.0,739.0,2018-06-13 12:19:38,0


In [12]:
# data to Regression
datasetNumeric = transform_output_to_numeric(datasetNumericClassification)

datasetNumeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  float32
 1   location            542339 non-null  float32
 2   timestamp           542339 non-null  float32
 3   time_to_next_alarm  542339 non-null  float32
 4   sound               542339 non-null  float32
 5   proximity           542339 non-null  float32
 6   phone_lock          542339 non-null  float32
 7   light               542339 non-null  float32
 8   day_of_week         542339 non-null  float32
 9   minutes_day         542339 non-null  float32
 10  timestamp_text      542339 non-null  object 
 11  class               542339 non-null  int32  
dtypes: float32(10), int32(1), object(1)
memory usage: 26.9+ MB


In [13]:
# save files
datasetNumeric.to_csv(outputFolders+"allData"+fileNumeric, sep=',', encoding='utf-8', index=False)

datasetNumeric

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
1,1.0,1.0,1.526390e+09,594000000.0,0.000000,1.0,0.0,37.0,1.0,860.0,2018-05-15 14:20:45,0
2,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,35.0,1.0,861.0,2018-05-15 14:21:15,0
3,2.0,2.0,1.526391e+09,593939968.0,243.576523,1.0,0.0,307.0,1.0,861.0,2018-05-15 14:21:45,0
4,2.0,2.0,1.526391e+09,593880000.0,242.545029,1.0,0.0,42.0,1.0,862.0,2018-05-15 14:22:15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,2.0,3.0,1.528888e+09,59999.0,0.000000,1.0,1.0,8.0,2.0,734.0,2018-06-13 12:14:37,0
542335,2.0,3.0,1.528889e+09,420000.0,0.000000,1.0,1.0,11.0,2.0,738.0,2018-06-13 12:18:08,0
542336,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,11.0,2.0,739.0,2018-06-13 12:19:08,0
542337,2.0,3.0,1.528889e+09,359999.0,0.000000,1.0,1.0,12.0,2.0,739.0,2018-06-13 12:19:38,0


In [14]:
# normalization to Regression
datasetNumericNormalized = transform_normalize_data(datasetNumeric)

datasetNumericNormalized.info()

activity 1.0 5
location 1.0 3
timestamp 1526043900.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0.0 1.0
phone_lock 0.0 1.0
light 0.0 211880.0
day_of_week 1.0 7
minutes_day 0.0 1439
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  float32
 1   location            542339 non-null  float32
 2   timestamp           542339 non-null  float32
 3   time_to_next_alarm  542339 non-null  float32
 4   sound               542339 non-null  float32
 5   proximity           542339 non-null  float32
 6   phone_lock          542339 non-null  float32
 7   light               542339 non-null  float32
 8   day_of_week         542339 non-null  float32
 9   minutes_day         542339 non-null  float32
 10  timestamp_text      542339 non-null  object 
 11  class               542

In [15]:
datasetNumericNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,0
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,0
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.000000,0.598332,2018-05-15 14:21:15,0
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.000000,0.598332,2018-05-15 14:21:45,0
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.000000,0.599027,2018-05-15 14:22:15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,0.25,1.0,0.007132,0.000099,0.000000,1.0,1.0,0.000038,0.166667,0.510076,2018-06-13 12:14:37,0
542335,0.25,1.0,0.007132,0.000694,0.000000,1.0,1.0,0.000052,0.166667,0.512856,2018-06-13 12:18:08,0
542336,0.25,1.0,0.007132,0.000595,0.000000,1.0,1.0,0.000052,0.166667,0.513551,2018-06-13 12:19:08,0
542337,0.25,1.0,0.007133,0.000595,0.000000,1.0,1.0,0.000057,0.166667,0.513551,2018-06-13 12:19:38,0


In [16]:
# save files
datasetNumericNormalized.to_csv(outputFolders+"allData"+fileNumericNormalized, sep=',', encoding='utf-8', index=False)

datasetNumericNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,0
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,0
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.000000,0.598332,2018-05-15 14:21:15,0
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.000000,0.598332,2018-05-15 14:21:45,0
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.000000,0.599027,2018-05-15 14:22:15,0
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,0.25,1.0,0.007132,0.000099,0.000000,1.0,1.0,0.000038,0.166667,0.510076,2018-06-13 12:14:37,0
542335,0.25,1.0,0.007132,0.000694,0.000000,1.0,1.0,0.000052,0.166667,0.512856,2018-06-13 12:18:08,0
542336,0.25,1.0,0.007132,0.000595,0.000000,1.0,1.0,0.000052,0.166667,0.513551,2018-06-13 12:19:08,0
542337,0.25,1.0,0.007133,0.000595,0.000000,1.0,1.0,0.000057,0.166667,0.513551,2018-06-13 12:19:38,0


In [17]:
# transform to classification and print
datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericNormalized)

datasetNumericClassificationNormalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 542339 entries, 0 to 542338
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            542339 non-null  float32
 1   location            542339 non-null  float32
 2   timestamp           542339 non-null  float32
 3   time_to_next_alarm  542339 non-null  float32
 4   sound               542339 non-null  float32
 5   proximity           542339 non-null  float32
 6   phone_lock          542339 non-null  float32
 7   light               542339 non-null  float32
 8   day_of_week         542339 non-null  float32
 9   minutes_day         542339 non-null  float32
 10  timestamp_text      542339 non-null  object 
 11  class               542339 non-null  object 
dtypes: float32(10), object(2)
memory usage: 29.0+ MB


In [18]:
# save files
datasetNumericClassificationNormalized.to_csv(outputFolders+"allData"+fileNumericClassificationNormalized, sep=',', encoding='utf-8', index=False)

datasetNumericClassificationNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.000000,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.000000,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.000000,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.000000,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
542334,0.25,1.0,0.007132,0.000099,0.000000,1.0,1.0,0.000038,0.166667,0.510076,2018-06-13 12:14:37,awake
542335,0.25,1.0,0.007132,0.000694,0.000000,1.0,1.0,0.000052,0.166667,0.512856,2018-06-13 12:18:08,awake
542336,0.25,1.0,0.007132,0.000595,0.000000,1.0,1.0,0.000052,0.166667,0.513551,2018-06-13 12:19:08,awake
542337,0.25,1.0,0.007133,0.000595,0.000000,1.0,1.0,0.000057,0.166667,0.513551,2018-06-13 12:19:38,awake


# Files by students

In [19]:
# Dataframe of all datasets
# initialize list of lists
datasets = [['0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs'],
            ['0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA'], 
            ['2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0'], 
            ['2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys'], 
            #['5FLZBTVAPwdq9QezHE2sVCJIs7p+r6mCemA2gp9jATk'], #does not have the file
            ['7EYF5I04EVqisUJCVNHlqn77UAuOmwL2Dahxd3cA'], 
            ['a9Qgj8ENWrHvl9QqlXcIPKmyGMKgbfHk9Dbqon1HQP4'], 
            ['ae4JJBZDycEcY8McJF+3BxyvZ1619y03BNdCxzpZTc'], 
            ['Ch3u5Oaz96VSrQbf0z31X6jEIbeIekkC0mwPzCdeJ1U'], 
            ['CH8f0yZkZL13zWuE9ks1CkVJRVrr+jsGdUXHrZ6YeA'], 
            ['DHO1K4jgiwZJOfQTrxvKE2vn7hkjamigroGD5IaeRc'], 
            ['DHPqzSqSttiba1L3BD1cptNJPjSxZ8rXxF9mY3za6WA'], 
            ['dQEFscjqnIlug8Tgq97JohhSQPG2DEOWJqS86wCrcY'], 
            ['HFvs2CohmhHte+AaCzFasjzegGzxZKPhkrX23iI6Xo'], 
            ['jgB9E8v3Z6PKdTRTCMAijBllA9YEMtrmHbe4qsbmJWw'], 
            ['JkY++R7E8myldLN3on6iQ78Ee78zCbrLuggfwGju3I'], 
            ['K4SLohf+TN1Ak8Dn8iE3Lme7rEMPISfppB2sXfHX8'], 
            ['oGaWetJJJEWHuvYdWYo826SQxfhCExVVQ2da8LE1Y7Q'], 
            ['pyt24oiDAHsmgWMvkFKz2fn2pwcHiXchd6KchLM'], 
            ['PZCf1nfvhR+6fk+7+sPNMYOgb8BAMmtQtfoRS83Suc'], 
            ['QUNCATForxzK0HHw46LrGOMWh0eVA8Y5XWEiUXX+cQ'], 
            ['rIl2UK9+bQ+tzpFdbJAdbBxEa5GbgrgC030yEaENLw'], 
            ['RoBW3cDOO9wWRMPO2twQff83MPc+OXn6gJ+a1DafreI'], 
            ['SH3kQeyd5volraxw8vOyhlowNqWBPr1IJ9URNXUL4'], 
            ['VVpwFNMrEglveh6MDN8lrRzTy5OwzglD4FURfM4A2is'], 
            ['Wa1mcNmbh66S7VS6GIzyfCFMD3SGhbtDQyFP1ywJEsw'], 
            ['XCKRE0BWRHxfP1kZIihgtT+jUjSp2GE8v5ZlhcIhVmA'], 
            ['YI5Y79K6GXqAUoGP6PNyII8WKlAoel4urDxWSVVOvBw'], 
            ['ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM'], 
            ['ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY']]
  
# Create the pandas DataFrame
folders = pd.DataFrame(datasets, columns=['folder'])

#folders

In [20]:
for indexFolder, folder in folders.iterrows():
    dataset = pd.read_csv(outputFolders+"student_"+folder['folder']+".csv")
    print()
    print("student_"+str(folder['folder'])+".csv", Counter(dataset['class']) )
    # basic transformations 
    datasetNumericClassification = transform_data(dataset)
    datasetNumeric = transform_output_to_numeric(datasetNumericClassification)
    datasetNumeric = transform_data_type(datasetNumeric)
    # numeric used by the algorithm -> prediction
    datasetNumericNormalized = transform_data_type(datasetNumericClassification)
    datasetNumericNormalized = transform_normalize_data(datasetNumericNormalized)
    datasetNumericNormalized.to_csv(outputFolders+"student_"+folder['folder']+"_numeric.csv", sep=',', encoding='utf-8', index=False)
    # numeric used by the algorithm -> classification
    datasetNumericClassificationNormalized = transform_output_to_str(datasetNumericNormalized)
    datasetNumericClassificationNormalized.to_csv(outputFolders+"student_"+folder['folder']+"_nominal.csv", sep=',', encoding='utf-8', index=False)


student_0Jf4TH9Zzse0Z1Jjh7SnTOe2MMzeSnFi7feTnkG6vgs.csv Counter({'awake': 16341, 'asleep': 1652})
activity 1.0 5
location 1.0 3
timestamp 1526390400.0 1924905600.0
time_to_next_alarm 0.0 604800000.0
sound 0.0 403
proximity 0.0 1.0
phone_lock 0.0 1.0
light 0.0 60000.0
day_of_week 1.0 7
minutes_day 0.0 1439

student_0tdmm6rwW3KquQ73ATYYJ5JkpMtvbppJ0VzA2GExdA.csv Counter({'awake': 8158, 'asleep': 3403})
activity 1.0 5
location 1.0 3
timestamp 1526415200.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0.0 0.0
phone_lock 0.0 1.0
light 0.0 0.0
day_of_week 1.0 7
minutes_day 0.0 1439

student_2cyV53lVyUtlMj0BRwilEWtYJwUiviYoL48cZBPBq0.csv Counter({'awake': 2064, 'asleep': 1319})
activity 1.0 5
location 1.0 3
timestamp 1526390500.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0.0 0.0
phone_lock 0.0 1.0
light 0.0 0.0
day_of_week 1.0 7
minutes_day 0.0 1439

student_2J22RukYnEbKTk7t+iUVDBkorcyL5NKN6TrLe89ys.csv Counter({'awake': 16586, 'aslee


student_ypklj+8GJ15rOIH1lpKQtFJOuK+VdvyCuBPqhY3aoM.csv Counter({'asleep': 36, 'awake': 23})
activity 1.0 5
location 1.0 3
timestamp 1526390300.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0.0 1.0
phone_lock 0.0 1.0
light 0.0 586.0
day_of_week 1.0 7
minutes_day 725.0 1439

student_ZSsAZ0Pq+MCqFrnjsRFn5Ua09pMCVaOV9c8ZuYb7XQY.csv Counter({'awake': 18781, 'asleep': 4971})
activity 1.0 5
location 1.0 3
timestamp 1526570500.0 1924905600.0
time_to_next_alarm -1.0 604800000.0
sound 0.0 403
proximity 0.0 1.0
phone_lock 0.0 1.0
light 0.0 33895.0
day_of_week 1.0 7
minutes_day 0.0 1439


In [21]:
datasetNumericNormalized

,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000000e+00,0.974504,0.000000,1.0,0.0,0.004956,0.333333,0.681723,2018-05-17 16:21:34,awake
1,0.00,0.5,0.000000e+00,0.974405,0.604264,1.0,0.0,0.002154,0.333333,0.682418,2018-05-17 16:22:34,awake
2,0.25,0.5,3.213375e-07,0.974306,0.597445,1.0,0.0,0.003629,0.333333,0.683113,2018-05-17 16:23:04,awake
3,0.00,0.5,3.213375e-07,0.974306,0.597445,1.0,0.0,0.002213,0.333333,0.683113,2018-05-17 16:23:34,awake
4,0.25,0.5,3.213375e-07,0.974206,0.536104,1.0,0.0,0.003157,0.333333,0.683808,2018-05-17 16:24:34,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
23747,0.25,1.0,5.819100e-03,0.000099,0.000000,1.0,1.0,0.000236,0.166667,0.510076,2018-06-13 12:14:37,awake
23748,0.25,1.0,5.819743e-03,0.000694,0.000000,1.0,1.0,0.000325,0.166667,0.512856,2018-06-13 12:18:08,awake
23749,0.25,1.0,5.819743e-03,0.000595,0.000000,1.0,1.0,0.000325,0.166667,0.513551,2018-06-13 12:19:08,awake
23750,0.25,1.0,5.820064e-03,0.000595,0.000000,1.0,1.0,0.000354,0.166667,0.513551,2018-06-13 12:19:38,awake
